Misc blocks of code to write out odds-and-ends linux commands for HPC

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
google_root = "Q:"
data_path = r"\Shared drives\Pandemic Data"
model_name = "slf_model"
run_name = "slf_grid_broad" # folder of your grid search 
total_runs = 80 # Count of runs expected (this is only needed if you had variable #'s of runs - ie. from two rounds of parameter sampling)

data_dir = f"{google_root}{data_path}\{model_name}"

In [ ]:
stats_dir = f"{data_dir}/outputs/summary_stats/{run_name}"

In [ ]:
stats = pd.read_csv(f"{stats_dir}/summary_stats_wPrecisionRecallF1FBetaAggProb.csv")
# Only needed if you have variable numbers of runs (to get the set you are looking for...)

stats = stats.groupby("sample").filter(lambda x: len(x) == total_runs)
agg_dict = {
    "start":["max"],
    "alpha":["max"],
    "lamda": ["max"],
    "count_known_countries_time_window_fbeta": ["mean","std"]
}

agg_dict = {**agg_dict}

agg_df = stats.groupby("sample").agg(agg_dict)

agg_df.columns = ["_".join(x) for x in agg_df.columns.values]
agg_df = agg_df.rename(columns={"start_max":"start","alpha_max":"alpha","lamda_max":"lamda","count_known_countries_time_window_fbeta_mean":"fbeta"})
agg_df['st_err']=agg_df['count_known_countries_time_window_fbeta_std']/np.sqrt(50)

In [ ]:
# Top 20 runs
top20 = agg_df.sort_values('fbeta',ascending=False).head(20).reset_index()

In [ ]:
# Copying the first 80 runs to the 200 runs folder
# folder structure year2006_alpha0.7_lamda2.2_6802-6802

command_string = "cp -r "

for sample_name in top20["sample"]:
    command_string += f'year{top20.loc[top20["sample"]==sample_name, "start"].values[0]}_alpha{top20.loc[top20["sample"]==sample_name, "alpha"].values[0]}_lamda{top20.loc[top20["sample"]==sample_name, "lamda"].values[0]}_6802-6802 '

command_string += "../slf_grid_top20"

print(command_string)

In [ ]:
# COPYING 200 runs to 1000 runs folders

# folder structure year2006_alpha0.7_lamda2.2_6802-6802
command_string = ""

for sample_name in top20["sample"][0:5]:
    command_string += f'cp -r slf_origin/outputs/slf_grid_top20/year{top20.loc[top20["sample"]==sample_name, "start"].values[0]}_alpha{top20.loc[top20["sample"]==sample_name, "alpha"].values[0]}_lamda{top20.loc[top20["sample"]==sample_name, "lamda"].values[0]}_6802-6802/. slf_top5/outputs/slf_top5_1000/year{top20.loc[top20["sample"]==sample_name, "start"].values[0]}_alpha{top20.loc[top20["sample"]==sample_name, "alpha"].values[0]}_lamda{top20.loc[top20["sample"]==sample_name, "lamda"].values[0]}_6802-6802/\n'

print(command_string)

In [ ]:
# Commands - runs 80 to 200 
start_run = 80 
end_run = 200
commands = ""

script = "python model_run_args.py"

for i in top20.index:
    commands += (
        " ".join(
            [
                script,
                str(top20['alpha'][i]),
                str(top20['lamda'][i]),
                str(top20['start'][i]),
                str(start_run),
                str(end_run),
            ]
        )
        + "\n"
    )

print(commands)

In [ ]:
# 1000 commands in batches of 20

commands = ""

for i in list(range(0,5)):
    start_run = 201 
    end_run = start_run + 20

    script = "python model_run_args.py"
    
    for j in list(range(0,40)):
        commands += (
            " ".join(
                [
                    script,
                    str(top20['alpha'][i]),
                    str(top20['lamda'][i]),
                    str(top20['start'][i]),
                    str(start_run),
                    str(end_run),
                ]
            )
            + "\n"
        )
        start_run = end_run+1
        end_run += 20

print(commands)
